# pyiron

## Define workflow with pyiron_base

In [1]:
from pyiron_base import job

In [2]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [3]:
from arithmetic_workflow import (
    get_sum as _get_sum,
    get_prod_and_div as _get_prod_and_div,
)

In [4]:
workflow_json_filename = "pyiron_arithmetic.json"

In [5]:
get_sum = job(_get_sum)
get_prod_and_div = job(_get_prod_and_div, output_key_lst=["prod", "div"])

In [7]:
obj = get_prod_and_div(x=1, y=2)

In [8]:
w = get_sum(x=obj.output.prod, y=obj.output.div)

In [9]:
write_workflow_json(delayed_object=w, file_name=workflow_json_filename)

In [10]:
!cat {workflow_json_filename}

{"nodes": [{"id": 0, "function": "simple_workflow.add_x_and_y"}, {"id": 1, "function": "simple_workflow.get_prod_and_div"}, {"id": 2, "value": 1}, {"id": 3, "value": 2}], "edges": [{"target": 0, "targetPort": "x", "source": 1, "sourcePort": "prod"}, {"target": 1, "targetPort": "x", "source": 2, "sourcePort": null}, {"target": 1, "targetPort": "y", "source": 3, "sourcePort": null}, {"target": 0, "targetPort": "y", "source": 1, "sourcePort": "div"}]}

## Load Workflow with aiida

In [11]:
from aiida import load_profile

load_profile()

Profile<uuid='9da6a68ed2b14ffb920b92aa0a616339' name='pwd'>

In [12]:
from python_workflow_definition.aiida import load_workflow_json

In [13]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [14]:

wg.run()

04/12/2025 11:28:38 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_prod_and_div2
04/12/2025 11:28:39 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|update_task_state]: Task: get_prod_and_div2, type: PyFunction, finished.
04/12/2025 11:28:39 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y1
04/12/2025 11:28:40 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|update_task_state]: Task: add_x_and_y1, type: PyFunction, finished.
04/12/2025 11:28:40 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
04/12/2025 11:28:40 AM <641> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15|WorkGraphEngine|finalize

## Load Workflow with jobflow

In [15]:
from python_workflow_definition.jobflow import load_workflow_json

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [16]:
from jobflow.managers.local import run_locally

In [17]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [18]:
result = run_locally(flow)
result

2025-04-12 11:28:41,600 INFO Started executing jobs locally
2025-04-12 11:28:42,179 INFO Starting job - get_prod_and_div (a00c2b77-12e1-4ec7-9919-99f65db51848)
2025-04-12 11:28:42,184 INFO Finished job - get_prod_and_div (a00c2b77-12e1-4ec7-9919-99f65db51848)
2025-04-12 11:28:42,186 INFO Starting job - add_x_and_y (5e71d195-b5a0-4eda-9279-0587f97ad019)
2025-04-12 11:28:42,190 INFO Finished job - add_x_and_y (5e71d195-b5a0-4eda-9279-0587f97ad019)
2025-04-12 11:28:42,191 INFO Finished executing jobs locally


{'a00c2b77-12e1-4ec7-9919-99f65db51848': {1: Response(output={'prod': 2, 'div': 0.5}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '5e71d195-b5a0-4eda-9279-0587f97ad019': {1: Response(output=2.5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))}}